In [1]:
import sys
from download_utils import download_week2_resources
from itertools import chain
import numpy as np

In [2]:
download_week2_resources()

File data/train.txt is already downloaded.
File data/validation.txt is already downloaded.
File data/test.txt is already downloaded.


In [3]:
def read_data(file_path):
    import re
    
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            token = re.sub('(http[s]*:\/\/[^\s]+)', 'URL', token)
            token = re.sub('(@[^\s]+)', 'USR', token)
            
            tweet_tokens.append(token)
            tweet_tags.append(tag)
    
    return tokens, tags                        

In [4]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

In [5]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
USR	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
URL	O
URL	O

Happy	O
Birthday	O
USR	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



In [6]:
from collections import defaultdict

In [7]:
def build_dict(tokens_or_tags, special_tokens):    
    
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    #remove duplicateds
    special_tokens_set = set(special_tokens)
    tokens_or_tags_set = set(chain.from_iterable(tokens_or_tags))
    valid_tokens_or_tags = tokens_or_tags_set - special_tokens_set
    
    idx2tok = list(special_tokens_set) + list(valid_tokens_or_tags)
    for index, token in enumerate(idx2tok):
        tok2idx[token] = index
        
    return tok2idx, idx2tok

In [8]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

In [9]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

**Generate batches**

In [10]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        #cria numpy com pads
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths           

In [11]:
import tensorflow as tf

In [12]:
class BiLSTMModel():
    pass

In [13]:
def declare_paceholders(self):
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name="input_batch")
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name="ground_truth_tags")
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name="lengths")
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name="dropout_ph")

In [14]:
BiLSTMModel.__declare_placeholders = classmethod(declare_paceholders)

In [15]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim)/np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(dtype=tf.float32, initial_value=initial_embedding_matrix, name="embeddings_matrix")
    
    lstm_forward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn)
    forward_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_forward_cell,
                                                 input_keep_prob=self.dropout_ph, 
                                                 output_keep_prob=self.dropout_ph,
                                                 state_keep_prob=self.dropout_ph)
    
    lstm_backward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden_rnn)
    backward_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_backward_cell,
                                                  input_keep_prob=self.dropout_ph,
                                                  output_keep_prob=self.dropout_ph,
                                                  state_keep_prob=self.dropout_ph)
    
    # Shape: [batch_size, sequence_len, embedding_dim]
    #tranforma minha seuqencia de indices de palavras em sequencia de embeddings
    embeddings = tf.nn.embedding_lookup(embedding_matrix_variable, self.input_batch)
    #passar os embedding para a bi-lstm
    (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(forward_cell,
                                                                        backward_cell,
                                                                        inputs=embeddings,
                                                                        sequence_length=self.lengths,
                                                                        dtype=tf.float32)
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)

In [16]:
BiLSTMModel.__build_layers = classmethod(build_layers)

In [17]:
def compute_predictions(self):
    softmax_output = tf.nn.softmax(self.logits)
    self.predictions = tf.argmax(softmax_output, axis=-1)

In [18]:
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [19]:
def compute_loss(self, n_tags, PAD_index):
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor = tf.nn.softmax_cross_entropy_with_logits(labels=ground_truth_tags_one_hot, 
                                                          logits=self.logits)
    mask = tf.cast(tf.not_equal(loss_tensor, PAD_index), tf.float32) #loss dos pads
    self.loss = tf.reduce_mean(tf.multiply(loss_tensor, mask))

In [20]:
BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [21]:
def perform_optimization(self):
    self.optimizer = tf.train.AdamOptimizer(self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    clip_norm = tf.cast(1.0, tf.float32)
    
    #aplicar clipping apenas no gradiente
    self.grads_and_vars = [(None, var) if grad is None else (tf.clip_by_norm(grad, clip_norm), var)
                          for grad, var in self.grads_and_vars]
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

In [22]:
BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [23]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

In [24]:
BiLSTMModel.__init__ = classmethod(init_model)

Treinamento

In [25]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {
        self.input_batch: x_batch,
        self.ground_truth_tags: y_batch,
        self.learning_rate_ph: learning_rate,
        self.dropout_ph: dropout_keep_probability,
        self.lengths: lengths
    }
    session.run(self.train_op, feed_dict=feed_dict)

In [26]:
BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [27]:
def predict_for_batch(self, session, x_batch, lengths):
    feed_dict={self.input_batch:x_batch,self.lengths:lengths}
    predictions=session.run(self.predictions,feed_dict=feed_dict)
    return predictions

In [28]:
BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

Avaliaçao

In [29]:
from evaluation import precision_recall_f1

In [30]:
def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
        
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

Execução

In [31]:
tf.reset_default_graph()

model = BiLSTMModel(len(idx2token),len(idx2tag),200,200,token2idx['<PAD>'])

batch_size = 32
n_epochs = 5
learning_rate = 0.007
learning_rate_decay = np.sqrt(2)
dropout_keep_probability = 0.5

In [32]:
%%time

sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_tokens, train_tags, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_tokens, validation_tags, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_tokens, train_tags):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 78604 phrases; correct: 215.

precision:  0.27%; recall:  4.79%; F1:  0.52

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 9474 phrases; correct: 22.

precision:  0.23%; recall:  4.10%; F1:  0.44

-------------------- Epoch 2 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 2764 phrases; correct: 996.

precision:  36.03%; recall:  22.19%; F1:  27.46

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 220 phrases; correct: 79.

precision:  35.91%; recall:  14.71%; F1:  20.87

-------------------- Epoch 3 of 5 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4614 phrases; correct: 2568.

precision:  55.66%; recall:  57.21%; F1:  56.42

Validation data evaluation:
processed 12836 tokens with 537 p